In [2]:
import pandas as pd
import numpy as np

link='https://en.wikipedia.org/wiki/List_of_American_films_of_2021'

df1=pd.read_html(link,header=0)[2]
df2=pd.read_html(link,header=0)[3]
df3=pd.read_html(link,header=0)[4]
df4=pd.read_html(link,header=0)[5]

df=df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

df=df.drop(['Opening','Opening.1','Production company','.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.','Ref.'],axis=1)

df.rename(columns={'Title':'movie_title'},inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_23848\315631984.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [3]:
df

,movie_title,Cast and crew
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...
3,The Dig,Simon Stone (director); Moira Buffini (screenp...
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa..."
...,...,...
356,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...
357,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...
358,American Underdog,"Erwin brothers (directors); Jon Erwin, David A..."
359,Memoria,Apichatpong Weerasethakul (director/acreenplay...


In [4]:
#director
df['director_name']=df['Cast and crew'].str.split('(',expand=True)[0]

In [7]:
#actors
df['actors']=df['Cast and crew'].str.split(';',expand=True)[1]
df['actors1']=df['Cast and crew'].str.split(';',expand=True)[2]

df['actors1']=df['actors1'].fillna('Unknown')

index=list(df[df['actors1']=='Unknown'].index)

for i in index:
    df['actors1'][i]=df['actors'][i]
    
df=df.drop('actors',axis=1)

df['actor_1_name']=df['actors1'].str.split(',',expand=True)[0]
df['actor_2_name']=df['actors1'].str.split(',',expand=True)[1]
df['actor_3_name']=df['actors1'].str.split(',',expand=True)[2]

df=df.drop(['actors1','Cast and crew'],axis=1)

In [10]:
df.fillna('Unknown',inplace=True)

In [11]:
df=df[df['movie_title']!='Unknown']

In [13]:
df['movie_title'].unique()

array(['Shadow in the Cloud', 'The White Tiger', 'Locked Down', 'The Dig',
       'Outside the Wire', 'The Marksman',
       'The Ultimate Playlist of Noise', 'American Skin',
       "Don't Tell a Soul", 'Our Friend', "No Man's Land",
       'Born a Champion', 'Wrong Turn', 'Penguin Bloom',
       'The Little Things', 'Finding ʻOhana', 'Nomadland',
       'Malcolm & Marie', 'Palmer', 'Falling', 'Bliss', 'Little Fish',
       'Son of the South', 'PVT Chat', 'Music',
       'Judas and the Black Messiah', 'The Mauritanian',
       'Breaking News in Yuba County',
       'Barb and Star Go to Vista Del Mar', 'Minari',
       'To All the Boys: Always and Forever', 'The World to Come',
       'French Exit', 'Land', "Willy's Wonderland",
       'The Map of Tiny Perfect Things', 'Fear of Rain', 'I Care a Lot',
       'Flora & Ulysses', 'Silk Road', 'Tom & Jerry',
       'The United States vs. Billie Holiday', 'Cherry', 'Crisis',
       "Billie Eilish: The World's a Little Blurry", 'The Vigil', '

In [14]:
#genre
from tmdbv3api import TMDb
import requests
import json
from tmdbv3api import Movie
tmdb=TMDb()
tmdb.api_key='e770358dbe349592b1fba6cb607230fb'

from tmdbv3api import Movie
tmdb_movie=Movie()
def get_genre(x):
    genres=[]
    result=tmdb_movie.search(x)
    movie_id=result[0].id
    response=requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json=response.json()
    if data_json['genres']:
        genre_str=' '
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            np.NaN
            
df['genres']=df['movie_title'].map(lambda x:get_genre(str(x)))

C:\Users\HP\AppData\Local\Temp\ipykernel_23848\4016790396.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres']=df['movie_title'].map(lambda x:get_genre(str(x)))


In [15]:
df=df[['movie_title','genres','director_name','actor_1_name','actor_2_name','actor_3_name']]

df['movie_title']=df['movie_title'].str.lower()

df['comb']=df['actor_1_name']+' '+df['actor_2_name']+' '+df['actor_3_name']+' '+df['director_name']+' '+df['genres']

In [16]:
data=pd.read_csv('movie_data.csv')

data=data.drop('Unnamed: 0',axis=1)

data=data.append(df,ignore_index=True)

data.to_csv('movie_data.csv')

C:\Users\HP\AppData\Local\Temp\ipykernel_23848\450147950.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df,ignore_index=True)
